# 5-1 卷積神經網路 CNN

In [67]:
# import os
# import numpy as np

# base_dir = r'/Users/cheng/Desktop/vscode/deep_learning/hw5.2_data/cats_and_dogs_small'
# train_dir = os.path.join(base_dir, 'train')
# validation_dir = os.path.join(base_dir, 'validation')
# test_dir = os.path.join(base_dir, 'test')
#像素值( 0和255之間)重新縮放到[0,1]間隔,進行正規化,方便deep learning做運算
# datagen = ImageDataGenerator(rescale=1./255)
# batch_size = 20

# def extract_features(directory, sample_count): #副函式,輸入是路徑(directory)跟樣本數(sample_count)
#     features = np.zeros(shape=(sample_count, 4, 4, 512)) #初始化,4*4像素的矩陣,有512個
#     labels = np.zeros(shape=(sample_count))
#     generator = datagen.flow_from_directory(directory, #把圖檔從路徑(directory)讀進來
#     target_size=(150, 150),
#     batch_size=batch_size,
#     class_mode='binary')

#     i = 0
#     for inputs_batch, labels_batch in generator: #萃取
#         features_batch = conv_base.predict(inputs_batch)
#         features[i * batch_size : (i + 1) * batch_size] = features_batch
#         labels[i * batch_size : (i + 1) * batch_size] = labels_batch
#         i += 1
#         print(i, end=' ') # 由於萃取需要較長的時間,我們印出 i 來檢視進度
#         if i * batch_size >= sample_count: # 當萃取樣本數超過設定的樣本數要中斷處理
#             break
#     return features, labels
# train_features, train_labels = extract_features(train_dir, 2000)
# validation_features, validation_labels = extract_features(validation_dir, 1000)
# test_features, test_labels = extract_features(test_dir, 1000)

In [68]:
# from keras import models
# from keras import layers
# from keras import optimizers
# model = models.Sequential()
# model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
# model.add(layers.Dropout(0.5)) # 丟棄法
# model.add(layers.Dense(1, activation='sigmoid'))
# #重新編譯
# model.compile(optimizer=optimizers.RMSprop(lr=2e-5), #優化器用RMSprop,微調所以學習率設很低

# loss='binary_crossentropy',
# metrics=['acc'])

# history = model.fit(train_features, # 用卷積基底萃取的特徵資料進行訓練與驗證

# train_labels,epochs=30,
# batch_size=20,
# validation_data=(validation_features, validation_labels))

In [69]:
# from keras import models
# from keras import layers
# from keras.applications import VGG16
# conv_base = VGG16(weights='imagenet',
# include_top=False,
# input_shape=(150, 150, 3))

# model = models.Sequential()
# model.add(conv_base) # 將卷積基底視為層加入 Sequential 模型中
# model.add(layers.Flatten()) # 攤平
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid')) # 增加密集層分類器
# model.summary() # 查看模型摘要

In [70]:
from keras import models
from keras import layers
from keras.applications import VGG16
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
conv_base.summary()
conv_base.trainable = True # 權重解凍
set_trainable = False
# 將所有層凍結到指定層為止
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

model = models.Sequential()
model.add(conv_base) # 將卷積基底視為層加入 Sequential 模型中
model.add(layers.Flatten()) # 攤平
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid')) # 增加密集層分類器
model.summary() # 查看模型摘要

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_18 (InputLayer)     │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 150, 150, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 150, 150, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 75, 75, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 75, 75, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 75, 75, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 37, 37, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 37, 37, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 18, 18, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 18, 18, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 9, 9, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ ?                      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 7,079,424 (27.01 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

In [71]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
train_datagen = ImageDataGenerator( #擴充訓練資料
rescale=1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255) # 請注意驗證資料不應該擴充

In [72]:
import os

base_dir = r'/Users/cheng/Documents/GitHub/Deep_Learning/0509/hw5.2_data/cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_generator = train_datagen.flow_from_directory(
train_dir, # 目標目錄路徑
target_size=(150, 150), # 調整所有圖像大小成 150×150
batch_size=20,
class_mode='binary')

# 因為使用二元交叉熵 binary_crossentropy 作為損失分數,所以需要二元標籤
validation_generator = test_datagen.flow_from_directory(
validation_dir,
target_size=(150, 150),
batch_size=20,
class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [75]:
# 編譯模型
model.compile(
loss='binary_crossentropy',
optimizer=optimizers.RMSprop(learning_rate=1e-5),
metrics=['acc'])
# 訓練模型
history = model.fit(
train_generator,
# steps_per_epoch=100,
epochs=100,
validation_data=validation_generator,
# validation_steps=50
)

Epoch 1/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 78s 765ms/step - acc: 0.8387 - loss: 0.3395 - val_acc: 0.9060 - val_loss: 0.2170
Epoch 2/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 78s 773ms/step - acc: 0.8716 - loss: 0.2867 - val_acc: 0.9120 - val_loss: 0.2088
Epoch 3/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 76s 762ms/step - acc: 0.8747 - loss: 0.2856 - val_acc: 0.9210 - val_loss: 0.1985
Epoch 4/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 78s 774ms/step - acc: 0.8902 - loss: 0.2579 - val_acc: 0.9150 - val_loss: 0.1886
Epoch 5/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 80s 797ms/step - acc: 0.9077 - loss: 0.2381 - val_acc: 0.9230 - val_loss: 0.1787
Epoch 6/100
 78/100 ━━━━━━━━━━━━━━━━━━━━ 12s 563ms/step - acc: 0.9048 - loss: 0.2297

KeyboardInterrupt: 

In [76]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label= 'Training acc') 
plt.plot(epochs, val_acc, 'b', label="Validation acc")
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

NameError: name 'history' is not defined

In [ ]:
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
def smooth_curve(points, factor=0.8):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

In [ ]:
plt.plot( epochs, smooth_curve(acc), 'bo', label='Smoothed training acc')
plt.plot( epochs, smooth_curve(val_acc), 'b', label='Smoothed validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot( epochs, smooth_curve(loss), 'bo', label='Smoothed training loss')
plt.plot( epochs, smooth_curve(val_loss), 'b', label='Smoothed validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
test_generator = test_datagen.flow_from_directory(
test_dir,
target_size=(150, 150),
batch_size=20,
class_mode='binary')
test_loss, test_acc = model.evaluate_generator(test_generator,steps=50)
print('test acc:', test_acc)